In [ ]:
#Алгоритм Эдмондса — Карпа
def edmonds_karp(graph, source, sink):
    # Создаём остаточную сеть
    residual = [{} for _ in range(len(graph))]
    for u in graph:
        for v, cap in graph[u].items():
            residual[u][v] = cap
            residual[v].setdefault(u, 0)  # Инициализация обратного ребра
    
    max_flow = 0
    
    while True:
        # Поиск кратчайшего пути через BFS
        parent = [-1] * len(graph)
        min_capacity = [float('inf')] * len(graph)
        queue = [source]
        parent[source] = source
        
        found_sink = False
        while queue and not found_sink:
            u = queue.pop(0)
            
            for v in residual[u]:
                capacity = residual[u][v]
                if capacity > 0 and parent[v] == -1:
                    parent[v] = u
                    min_capacity[v] = min(min_capacity[u], capacity)
                    
                    if v == sink:
                        found_sink = True
                        break  # Найден путь к стоку
                    queue.append(v)

        # Если пути до стока нет
        if parent[sink] == -1:
            return max_flow
            
        # Увеличиваем поток вдоль найденного пути
        flow = min_capacity[sink]
        max_flow += flow
        
        # Обновляем остаточные пропускные способности
        v = sink
        while v != source:
            u = parent[v]
            residual[u][v] -= flow
            residual[v][u] += flow
            v = u

In [ ]:
#алгоритм поиска минимального рёберного покрытия в двудольном графе
def kuhn(graph):
    match = {}
    right = set()
    for u in graph:
        for v in graph[u]:
            right.add(v)
    
    for v in right:
        match[v] = None
    
    def dfs(u, visited):
        for v in graph[u]:
            if v not in visited:
                visited.add(v)
                if match[v] is None or dfs(match[v], visited):
                    match[v] = u
                    return True
        return False
    
    for u in graph:
        dfs(u, set())
    
    return match

def minimal_edge_cover(graph):
    # 1. Находим максимальное паросочетание
    match = kuhn(graph)
    edge_cover = set()
    for v, u in match.items():
        if u is not None:
            edge_cover.add((u, v))
    
    # 2. Добавляем рёбра для непокрытых вершин
    left = set(graph.keys())
    right = set().union(*graph.values())
    
    # Непокрытые вершины левой доли
    uncovered_left = left - {u for u, _ in edge_cover}
    
    # Непокрытые вершины правой доли
    uncovered_right = right - {v for _, v in edge_cover}
    
    # Добавляем рёбра для левой доли
    for u in uncovered_left:
        for v in graph[u]:
            if v in uncovered_right:
                edge_cover.add((u, v))
                uncovered_right.remove(v)
                break
    
    # Добавляем рёбра для оставшихся вершин правой доли
    for v in uncovered_right:
        for u in left:
            if v in graph[u]:
                edge_cover.add((u, v))
                break
    
    return edge_cover
